In [2]:
import xmltodict

# Load XML from file or string
with open(r"simulation\SimulationExperimentTemplateTest\solution\Soil3_Germany_AllKreis.sol.xml") as f:
    xml_dict = xmltodict.parse(f.read())

In [3]:
xml_dict

{'solution': {'@check': 'LAZY',
  '@version': '4.2',
  'description': {'@title': 'USL',
   '@author': 'Sabine, Andi E.',
   '@qualitylevel': 'I',
   '#text': 'WIKI_START\n\t\t=== Purpose === Runs the cefit CKA solution ==='},
  'variables': {'var': [{'@id': 'spinupdays',
     '@datatype': 'INT',
     '#text': '0'},
    {'@id': 'vIOPT', '@datatype': 'INT', '#text': '4'},
    {'@id': 'vTRANRF', '@datatype': 'DOUBLE', '#text': '1.0'},
    {'@id': 'vCultivarGroup',
     '@datatype': 'CHAR',
     '@unit': '',
     '#text': 'winter wheat'},
    {'@id': 'vAverageGroundTemperature',
     '@datatype': 'DOUBLE',
     '#text': '9.51'},
    {'@id': 'vAverageBulkDensity', '@datatype': 'DOUBLE', '#text': '1.5'},
    {'@id': 'vFirstDayMeanTemp', '@datatype': 'DOUBLE', '#text': '2.28'},
    {'@id': 'vCarbonContent', '@datatype': 'DOUBLE', '#text': '0.77'},
    {'@id': 'vTopSoilDepth', '@datatype': 'DOUBLE', '#text': '0.3'},
    {'@id': 'vRunNr', '@datatype': 'INT', '#text': '1'},
    {'@id': 'vreldem'